In [1]:
%run setup/GpuOptions.ipynb

In [8]:
import os
import sys

sys.path.append('..')

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.layers import *
import common.model as m
from common.dataset import Dataset

dataset = Dataset(tf.keras.utils.image_dataset_from_directory(
    '../dataset/dataset-2022-10-19T22-51-08',
    batch_size=32,
    image_size=(1920, 1080),
))
augmentation = [
    RandomTranslation(0.05, 0.05, fill_mode='nearest'),
    RandomBrightness(0.025),
    Cropping2D(cropping=((160, 730), (580, 1470))),
]

classes = dataset.original.class_names
data, _ = next(iter(dataset.training))
shape = data[0].shape
inputs = tf.keras.Input(shape=shape)

outputs = m.chained(
    # Augmentations
    *augmentation,
    # Entry Flow
    tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=2, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Conv2D(filters=64, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    m.expand(
        flow=lambda previous_activation, size: tf.keras.layers.add([
            m.chained(
                m.duplicate(
                    layers=lambda: [
                        tf.keras.layers.Activation('relu'),
                        tf.keras.layers.SeparableConv2D(filters=size, kernel_size=3, padding='same'),
                        tf.keras.layers.BatchNormalization(),
                    ],
                    count=2
                ),
                tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
            )(previous_activation),
            tf.keras.layers.Conv2D(filters=size, kernel_size=1, strides=2, padding='same')(previous_activation),
        ]),
        values=[128, 256, 728],
    ),

    # # Middle Flow
    m.expand(
        flow=lambda previous_activation, _: tf.keras.layers.add([
            m.duplicate(
                layers=lambda: [
                    tf.keras.layers.Activation('relu'),
                    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
                    tf.keras.layers.BatchNormalization(),
                ],
                count=3,
            )(previous_activation),
            previous_activation,
        ]),
        values=[0] * 8
    ),

    # # Exit Flow
    lambda previous_activation: tf.keras.layers.add([
        m.chained(
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.Activation('relu'),
            tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
            tf.keras.layers.BatchNormalization(),
            tf.keras.layers.MaxPooling2D(pool_size=3, strides=2, padding='same'),
        )(previous_activation),
        tf.keras.layers.Conv2D(filters=1024, kernel_size=1, strides=2, padding='same')(previous_activation),
    ]),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=728, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.SeparableConv2D(filters=1024, kernel_size=3, padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(len(classes), activation='linear'),
    tf.keras.layers.Dropout(0.2),
)(inputs)

model = tf.keras.Model(inputs=inputs, outputs=outputs)
model.build(input_shape=(None, *shape))
model.compile(
    optimizer=tf.keras.optimizers.Adadelta(learning_rate=0.08),
    loss=tf.keras.losses.CategoricalCrossentropy(
        from_logits=True, label_smoothing=0.03),
    metrics=['accuracy'],
)
if os.path.exists('isthemountainout.best.h5'):
    model.load_weights('isthemountainout.best.h5')
elif os.path.exists('isthemountainout.h5'):
    model.load_weights('isthemountainout.h5')
model.summary()

Found 3106 files belonging to 4 classes.


ValueError: Exception encountered when calling layer "cropping2d_6" (type Cropping2D).

Argument `cropping` must be greater than the input shape. Received: inputs.shape=(None, 1920, 1080, 3), and cropping=((160, 730), (580, 1470))

Call arguments received by layer "cropping2d_6" (type Cropping2D):
  • inputs=tf.Tensor(shape=(None, 1920, 1080, 3), dtype=float32)